In [4]:
import spacy
import pdfminer
import re
import os

In [5]:
import pandas as pd
# Force downgrade and ignore existing "wrong" versions
!pip install "numpy<2" --force-reinstall --user

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
streamlit 1.37.1 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.


In [6]:
import pdf2txt

In [7]:
def convert_pdf(f):
    output_filename=os.path.basename(os.path.splitext(f)[0])+".txt"
    output_filepath=os.path.join("output/txt/",output_filename)
    pdf2txt.main(args=[f, "--outfile", output_filepath])
    print(output_filepath+" saved succesfully!!!")
    return open(output_filepath).read()

In [8]:
os.path.splitext("prashant.pdf")

('prashant', '.pdf')

In [9]:
# load the language model
nlp=spacy.load("en_core_web_sm")

In [10]:
result_dict={'name':[], 'phone': [], 'email': [], 'skills': [] }
names=[]
phones=[]
emails=[]
skills=[]

In [11]:
def parse_content(text):
    skillset = re.compile("python|java|sql|hadoop|tableau")
    phone_num = re.compile(
        r"(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"
    )
    doc = nlp(text)
    name = [entity.text for entity in doc.ents if entity.label_ == "PERSON"][0]
    print(name)
    email = [word for word in doc if word.like_email == True][0]
    print(email)
    phone = str(re.findall(phone_num, text.lower()))
    skills_list = re.findall(skillset, text.lower())
    unique_skills_list = str(set(skills_list))
    names.append(name)
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Extraction completed successfully!!!")

In [12]:
for file in os.listdir('resume/'):
    if file.endswith('.pdf'):
        print('Reading....'+file)
        txt=convert_pdf(os.path.join('resume/',file))
        parse_content(txt)

Reading....Alisson ParkerCV.pdf
output/txt/Alisson ParkerCV.txt saved succesfully!!!
Alisson Parker-Wright                                                                                                 
alli1414parks@mail.com
Extraction completed successfully!!!
Reading....AshleyMilesCV.pdf
output/txt/AshleyMilesCV.txt saved succesfully!!!
Ashley Miles
ashleymiles@memail.com
Extraction completed successfully!!!
Reading....John DominicCV.pdf
output/txt/John DominicCV.txt saved succesfully!!!
John Dominic
johndominic@mail.com
Extraction completed successfully!!!


In [13]:
result_dict['name']=names
result_dict['phone']=phones
result_dict['email']=emails
result_dict['skills']=skills

In [14]:
result_df = pd.DataFrame(result_dict)
result_df

,name,phone,email,skills
0,Alisson Parker-Wright ...,['8569878511'],alli1414parks@mail.com,"{'java', 'python', 'tableau'}"
1,Ashley Miles,['6592251422'],ashleymiles@memail.com,"{'sql', 'tableau'}"
2,John Dominic,['7877756411'],johndominic@mail.com,"{'java', 'hadoop', 'python'}"


In [15]:

result_df.to_csv('output/csv/parsed_resumes.csv')